In [ ]:
# TODO
# Logic for prompting the LLM to generate a playbook to remediate the error

In [7]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.5/725.5 kB 166.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [17]:
import os
from openai import OpenAI

In [18]:
api_key = os.getenv("gemini_api_key")

In [19]:
llm_endpoint="https://generativelanguage.googleapis.com/v1beta/openai/"

# Initialise OpenAI Client
client = OpenAI(
    api_key=api_key,
    base_url=llm_endpoint
)

In [20]:
# TODO
# Logic to fetch error to parse as a prompt to LLM

# Read the content of the file into a variable
with open('error.txt', 'r') as file:
    error_event = file.read()

print(error_event)


In [21]:
user_prompt = f"""
The following error was retrieved from a Linux virtual machine:

{error_event}

Based on the error above, generate an Ansible playbook that remediates the issue. 
Only output the raw YAML of the playbook, without any markdown formatting, comments, or explanations.
"""

response = client.chat.completions.create(
    model="gemini-2.0-flash",
    messages=[
        {"role": "system", "content": "You are an assistant for writing Ansible playbooks. Reply only with the raw YAML content — do not include markdown formatting like ```yaml."},
        {
            "role": "user",
            "content": user_prompt
        }
    ]
)

# For Debugging
# print(response.choices[0].message.content)

In [22]:
# Extract the content from the LLM response
playbook_content = response.choices[0].message.content

# Write it to a YAML file
with open("remediation.yaml", "w") as f:
    f.write(playbook_content)

In [23]:
# Remove the markdown syntax
# Read the content from the file
with open("remediation.yaml", "r") as f:
    content = f.read().strip()

# Remove markdown code block if present
if content.startswith("```yaml"):
    content = content.replace("```yaml", "", 1).strip()
if content.endswith("```"):
    content = content.rsplit("```", 1)[0].strip()

# Write the cleaned content back to the same file
with open("remediation.yaml", "w") as f:
    f.write(content)

In [2]:
# Print the playbook content
with open("remediation.yaml", "r") as f:
    content = f.read().strip()
print(content)

---
- hosts: all
  become: true
  tasks:
    - name: Install httpd
      yum:
        name: httpd
        state: present
      notify:
        - Restart httpd

    - name: Create /etc/sysconfig/httpd if it does not exist
      file:
        path: /etc/sysconfig/httpd
        state: touch
        mode: '0644'
      when: "'No such file or directory' in error_message" # Replace error_message with the actual variable containing the error

    - name: Start httpd service
      systemd:
        name: httpd
        state: started
        enabled: true
      ignore_errors: true

    - name: Ensure firewalld is running
      service:
        name: firewalld
        state: started
        enabled: true

    - name: Open port 80 in firewall
      firewall:
        port: 80/tcp
        permanent: true
        state: enabled
      notify:
        - Reload firewalld

  handlers:
    - name: Restart httpd
      systemd:
        name: httpd
        state: restarted

    - name: Reload firewalld
     